In [16]:
def readfile(filename):
    lines = [line for line in file(filename)]
    
    #第一行是列标题
    colnames = lines[0].strip().split('\t')[1:]
    rownames = []
    data = []
    for line in lines[1:]:
        p = line.strip().split('\t')
        #每行的第一行是行名
        rownames.append(p[0])
        #剩余部分就是对应的数据
        data.append([float(x) for x in p[1:]])
    return rownames,colnames,data



In [32]:
#核心算法还是相关性
from math import sqrt
def pearson(v1,v2):
    sum1 = sum(v1)
    sum2 = sum(v2)
    
    sqare_sum1 = sum([pow(v,2)for v in v1])
    sqare_sum2 = sum([pow(v,2)for v in v2])
    
    pSum = sum( [v1[i]*v2[i] for i in range(len(v1))] )
    
    num = pSum - (sum1*sum2/len(v1))
    den = sqrt(sqare_sum1-pow(sum1,2)/len(v1))*(sqare_sum2-pow(sum2,2)/len(v1))
    if den == 0:return 0
    
    return 1.0-num/den

In [33]:
class bicluster( object ):
    def __init__(self,vec,left = None,right = None,distance = 0.0,id = None):
        self.left = left
        self.right = right
        self.vec = vec
        self.distance = distance
        self.id = id 
        

In [42]:
def hcluster(rows,distance = pearson):
    distances = {}
    currentclustid = -1
    
    #最开始的聚类就是数据集中的行
    clust = [bicluster(rows[i],id = i)for i in range(len(rows))]
    
    #终止条件直到只剩下一个终止条件为止
    while len(clust)>1:
        lowestpair = (0,1)
        closest = distance(clust[0].vec,clust[1].vec)
        
        #遍历每一个配对，寻找最小值
        for i in range(len(clust)):
            for j in range(i+1,len(clust)):
                #用distances缓存距离的计算值
                if (clust[i].id,clust[j].id) not in distances:
                    distances[clust[i].id,clust[j].id] = distance(clust[i].vec,clust[j].vec)
                    
                d = distance(clust[i].vec,clust[j].vec)
                if d < closest:
                    closest = d
                    lowestpair = (i,j)
                    
        #计算两个聚类的平均值
        mergevec = [(clust[lowestpair[0]].vec[i]+clust[lowestpair[1]].vec[i])/2.0 for i in range(len(clust[0].vec))]        
        
        #建立一个新的聚类
        newcluster = bicluster(mergevec,left = clust[lowestpair[0]],right=clust[lowestpair[1]],distance= closest,id = currentclustid)
        currentclustid -= 1
        del clust[lowestpair[1]]
        del clust[lowestpair[0]]
        clust.append(newcluster)
        
    return clust[0]

In [53]:
def printcluster(clust,labels = None,n = 0):
    #利用缩进来建立层级布局：
    for i in range(n):print ' ',
    if clust.id < 0: 
        #负数标记代表这是一个分支：
        print '-'
    else:
          #正数标记代表这是一个叶节点
        if labels == None: print clust.id 
        else:print labels[clust.id]
    
    #现在开始打印右侧分支和左侧分支：
    if clust.left != None:printcluster(clust.left,labels = labels,n = n+1)
    if clust.right != None:printcluster(clust.right,labels = labels,n = n+1)
        

 blognames,words,data = readfile('blogdata.txt')
clust = hcluster(data)


In [44]:
blognames,words,data = readfile('blogdata.txt')
clust = hcluster(data)


TypeError: object of type 'bicluster' has no len()

In [54]:
printcluster(clust,labels=blognames)

-
  SpikedHumor
  -
    -
      MAKE Magazine
      -
        Boing Boing
        -
          Topix.net Weblog
          Steve Pavlina's Personal Development Blog
    -
      -
        The Unofficial Apple Weblog (TUAW)
        -
          -
            Joystiq
            CoolerHeads Prevail
          -
            Download Squad
            -
              -
                Blog Maverick
                -
                  Techdirt
                  Creating Passionate Users
              -
                -
                  Kotaku
                  -
                    Gizmodo
                    TechEBlog
                -
                  Slashdot
                  -
                    GigaOM
                    -
                      -
                        Quick Online Tips
                        -
                          -
                            Official Google Blog
                            -
                              Google Blogoscoped
                   